In [43]:
from fractions import Fraction,  gcd
import numpy as np

    
def answer(m):
    """ Calculates the probabilities of reaching the terminal states"""

    # Get number of states.
    no_states = len(m)
    
    # Edge case 0: empty matrix.
    if (no_states == 0):
        print("Input matrix is empty")
        return []
        
    # Edge case 1: 1d matrix - Test 4 passed.
    if (no_states == 1):
        print("Input matrix is 1d")
        return [1, 1] # 0th state is final state for sure;)
    
    # Calculate tmp variable - sums of rows
    row_sums = [sum(i)  for i in m]
    #print("row_sums=", row_sums)
 
    # Get terminal states.
    terminal_states = []
    no_terminal = 0
    for i in range(no_states):
        # If there are no outputs.
        if (row_sums[i] == 0):
            terminal_states.append(True)
            no_terminal +=1
        # Or all outputs lead to the same node (diagonal):
        elif (len(m[i]) >i) and (row_sums[i] == m[i][i]) :
            terminal_states.append(True)
            no_terminal +=1
        else:
            terminal_states.append(False)
    print("terminal states ({}) = {}".format(no_terminal, terminal_states))

    # Edge case 2: no terminal states (task states clearly that this cannot happen, but just in case...)
    if (no_terminal == 0):
        print("There are no terminal states!")
        return []

    no_non_terminal = no_states - no_terminal
    # Edge case 3: all states are terminal states - which means that there are no transitions!
    # Edge case 1 is a special case of this edge case.
    if (no_non_terminal == 0):
        print("There are no non-terminal states!")
        res = [1] # 0-th state
        for _ in range(no_terminal-1):
            res.append(0)
        res.append(1) # denominator
        return res
    
    # Matrix A contains the probabilities of LEAVING FROM a given non-terminal state to a terminal state.
    A = []
    for i in range(no_states):
        # If the state is not terminal...
        if(not terminal_states[i]):
            # Diagonal matrix a.
            A.append([0 for _ in range(no_non_terminal)])
            # Sum of probabilities of leaving.
            #A[i][i] = sum([Fraction(int(a)*b/row_sums[i]).limit_denominator() for a,b in zip(terminal_states, m[i])])
            A[i][i] = sum([int(a)*b/row_sums[i] for a,b in zip(terminal_states, m[i])])

            
    # Matrix B contains the probabilities of GETTING TO another non-terminal state from a non-terminal state.
    B = []
    for _ in  range(no_non_terminal):
        B.append([0 for _ in range(no_non_terminal)])

    for i in range(no_states):
        # If the state is not terminal...
        if(not terminal_states[i]):
            b_index = 0
            for j in range(no_states):
                # Skip self-connection.
                if (i==j):
                    b_index += 1
                    continue
                # If the destination state is not terminal...
                if(not terminal_states[j]):
                    #B[b_index][i] = Fraction(m[i][j]/row_sums[i]).limit_denominator()
                    B[b_index][i] = m[i][j]/row_sums[i]
                    b_index += 1
    
            
    print("A=",A)    
    print("B=",B)    
    
    i_b = np.eye(no_non_terminal) - np.matrix(B) 
    print("i_b=",i_b)
    # Inverse!
    inv_i_b = i_b.I
    print("inv_i_b=",inv_i_b)
    
    axinv_i_b = A * i_b.I
    print("axinv_i_b=",axinv_i_b)
    
    AxiIB = []
    for _ in  range(no_non_terminal):
        AxiIB.append([0 for _ in range(no_non_terminal)])

    for i in range(no_non_terminal):
        for j in range(no_non_terminal):
            AxiIB[i][j] = Fraction(axinv_i_b[i,j]).limit_denominator()
    print("Frac=",AxiIB)
    
    X = [1, 0]
    
    print(AxiIB * X)
    # Return formated probabilities
    return 1


if __name__ == "__main__":
    ore_trans_mat = [
      [0,1,0,0,0,1],  # s0, the initial state, goes to s1 and s5 with equal probability
      [4,0,0,3,2,0],  # s1 can become s0, s3, or s4, but with different probabilities
      [0,0,0,0,0,0],  # s2 is terminal, and unreachable (never observed in practice)
      [0,0,0,0,0,0],  # s3 is terminalnumerators
      [0,0,0,0,0,0],  # s4 is terminal 
      [0,0,0,0,0,0],  # s5 is terminal
    ]
    
    #ore_trans_mat = [
    #    list(range(0,10)),
    #    list(range(1,11)),
    #    list(range(2,12)),
    #    list(range(3,13)),
    #    list(range(4,14)),
    #    [0, 0, 0, 0, 0, 1000, 0, 0, 0, 0],
    #    list(range(6,16)),
    #    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    #    list(range(8,18)),
    #    list(range(9,19))
    #]
    
    #ore_trans_mat = [
    #    [1, 0, 0, 0],
    #    [0, 1, 0, 0],
    #    [0, 0, 1, 0],
    #    [0, 0, 0, 1]
    #]
    
    #ore_trans_mat = [
    #    [1000, 2000, 3000, 4000],
    #    [0, 1000, 0, 0],
    #    [0, 0, 10001, 0],
    #    [0, 0, 0, 16000]
    #]
    
    
    #ore_trans_mat =   [[0, 2, 1, 0, 0], [0, 0, 0, 3, 4], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

    #ore_trans_mat = [[0, 1, 0, 0, 0, 1], [4, 0, 0, 3, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
    
    # Tricky cases!
    #ore_trans_mat =   [[], []]

    #ore_trans_mat =   [[0, 1,  0], [0, 1,  0],  [0, 1, 0]]

    #ore_trans_mat =   [[0,  2,  3,  4]]
    
    #ore_trans_mat =   [[0, 2], [1],  [0], [0, 0]]
    
    #ore_trans_mat =   [[1]]
    #ore_trans_mat =   [[0,  0],  [0, 1]]
    #ore_trans_mat = [[0,1,0,1], [1, 0, 0, 1], [0, 0, 0, 0], [0, 1, 1, 0]]
    
    #ore_trans_mat

    print("ore_trans_mat=",ore_trans_mat)

    print("answer =",answer(ore_trans_mat))

    

ore_trans_mat= [[0, 1, 0, 0, 0, 1], [4, 0, 0, 3, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
terminal states (4) = [False, False, True, True, True, True]
A= [[0.5, 0], [0, 0.5555555555555556]]
B= [[0, 0.4444444444444444], [0.5, 0]]
i_b= [[ 1.         -0.44444444]
 [-0.5         1.        ]]
inv_i_b= [[ 1.28571429  0.57142857]
 [ 0.64285714  1.28571429]]
axinv_i_b= [[ 0.64285714  0.28571429]
 [ 0.35714286  0.71428571]]
Frac= [[Fraction(9, 14), Fraction(2, 7)], [Fraction(5, 14), Fraction(5, 7)]]


TypeError: can't multiply sequence by non-int of type 'list'